In [0]:
#by. Rolly Vasquez
!pip install simpy

In [0]:
import simpy # módulo para realizar simulaciones
import random as rnd # módulo para generar números aleatorios
import numpy as np
# simulación basada en https://naps.com.mx/blog/simulacion-en-python-usando-simpy/

In [0]:
semilla = 30 # semilla para números aleatorios
n_servidores = 1 # cantidad de servidores en el sistema de cola de la caja de atención al cliente
tmin = 1 # parámetro de la distribución uniforme en minutos
tmax = 5 # parámetro de la distribución uniforme en minutos
t_llegadas = 6 # parámetro de la distribución exponencial en minutos
tot_cli = 15 # número máximo de clientes

tet = 0.0 # tiempo de espera total
dst = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

In [0]:
def atender(cliente):
	global dst  #Para poder acceder a la variable dst declarada anteriormente
	t_servicio = tmin + ((tmax - tmin)*rnd.random()) # valor aleatorio del tiempo de servicio de la distribución uniforme
	yield env.timeout(t_servicio) # deja correr el tiempo t_servicio minutos
	print(" \o/ Atención hecha %s en %.2f minutos" % (cliente,t_servicio))
	dst = dst + t_servicio # Acumula los tiempos de uso 

In [0]:
def cliente (env, nombre, personal ):
	global tet
	global fin
	llega = env.now # Guarda el minuto de llegada del cliente
	print ("---> %s llegó al banco en el minuto %.2f" % (nombre, llega))
	with personal.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa = env.now # Guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega # Calcula el tiempo que esperó
		tet = tet + espera # Acumula los tiempos de espera
		print ("**** %s pasa con el cajero en el minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
		yield env.process(atender(nombre)) # Invoca al proceso atender
		deja = env.now #Guarda el minuto en que termina el proceso atender 
		print ("<--- %s deja caja en minuto %.2f" % (nombre, deja))
		fin = deja # Conserva globalmente el último minuto de la simulación

In [0]:
def principal (env, personal):
	llegada = 0
	for i in range(tot_cli): # Para tot_cli clientes
		llegada = -t_llegadas * np.log(rnd.random()) # valor aleatorio del tiempo de llegada de la distribución exponencial
		yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
		env.process(cliente(env, 'Cliente %d' % i, personal))

In [0]:
print ("Simulación Banco")
rnd.seed(semilla)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulación de la caja de atención al cliente
personal = simpy.Resource(env, n_servidores) #Crea los recursos (servidores)
env.process(principal(env, personal)) #Invoca el proceso principal
env.run() #Inicia la simulación

Simulación Banco
---> Cliente 0 llegó al banco en el minuto 3.71
**** Cliente 0 pasa con el cajero en el minuto 3.71 habiendo esperado 0.00
 \o/ Atención hecha Cliente 0 en 1.12 minutos
<--- Cliente 0 deja caja en minuto 4.83
---> Cliente 1 llegó al banco en el minuto 11.15
**** Cliente 1 pasa con el cajero en el minuto 11.15 habiendo esperado 0.00
 \o/ Atención hecha Cliente 1 en 1.84 minutos
<--- Cliente 1 deja caja en minuto 12.99
---> Cliente 2 llegó al banco en el minuto 13.70
**** Cliente 2 pasa con el cajero en el minuto 13.70 habiendo esperado 0.00
 \o/ Atención hecha Cliente 2 en 2.59 minutos
<--- Cliente 2 deja caja en minuto 16.29
---> Cliente 3 llegó al banco en el minuto 21.85
**** Cliente 3 pasa con el cajero en el minuto 21.85 habiendo esperado 0.00
---> Cliente 4 llegó al banco en el minuto 24.51
 \o/ Atención hecha Cliente 3 en 4.96 minutos
<--- Cliente 3 deja caja en minuto 26.80
**** Cliente 4 pasa con el cajero en el minuto 26.80 habiendo esperado 2.29
---> Cliente 

In [0]:
print("\nIndicadores obtenidos: ")
lpc = tet/fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = tet/tot_cli
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dst/fin)/n_servidores
print ("Uso promedio de la instalacion = %.2f" % upi)


Indicadores obtenidos: 

Longitud promedio de la cola: 0.49
Tiempo de espera promedio = 2.27
Uso promedio de la instalacion = 0.65
